In [19]:
import pyspark
from pyspark import SparkContext
from datetime import datetime

In [3]:
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/amanmadan/opt/anaconda3/lib/python3.8/site-packages/IPython/utils/py3compat.py:168 

In [7]:
myRDD = sc.parallelize([('Mike', 19), ('June', 18), ('Rachel',16), ('Rob', 18), ('Scott', 17)])

In [8]:
myRDD.take(2)

[('Mike', 19), ('June', 18)]

In [9]:
myRDD = (sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/airport-codes-na.txt' 
                     , minPartitions=4
                    , use_unicode=True).map(lambda element: element.split("\t")))

In [12]:
myRDD.take(2)

[['City', 'State', 'Country', 'IATA'], ['Abbotsford', 'BC', 'Canada', 'YXX']]

In [13]:
myRDD.count()

527

In [14]:
myRDD.getNumPartitions()

4

In [15]:
depatureDelaysRDD = (sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv'
                                , minPartitions=4
                                , use_unicode=True).map(lambda x: x.split(",")))

In [33]:
a=datetime.now()
depatureDelaysRDD.count()
b=datetime.now()
print(b-a)


0:00:01.881323


4

In [34]:
depatureDelaysRDD.getNumPartitions()

4

In [36]:
depatureDelaysRDD_nonParition = (sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv'
                                , use_unicode=True).map(lambda x: x.split(",")))

In [32]:
a=datetime.now()
depatureDelaysRDD_nonParition.count()
b=datetime.now()
print(b-a)

0:00:01.896087


In [37]:
depatureDelaysRDD_nonParition.getNumPartitions()

2

In [ ]:
#RDD Transformations

In [38]:
airports = (sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/airport-codes-na.txt')
            .map(lambda row: row.split("\t")))

In [39]:
airports.take(5)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [40]:
flights = (sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv')
           .map(lambda row: row.split(",")))

In [42]:
flights.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [45]:
#Map Transformation
airports.map(lambda c: (c[0], c[1])).take(5)

[('City', 'State'),
 ('Abbotsford', 'BC'),
 ('Aberdeen', 'SD'),
 ('Abilene', 'TX'),
 ('Akron', 'OH')]

In [46]:
airports.map(lambda c: (c[0], c[1])).filter(lambda c: c[1]=='WA').take(5)

[('Bellingham', 'WA'),
 ('Moses Lake', 'WA'),
 ('Pasco', 'WA'),
 ('Pullman', 'WA'),
 ('Seattle', 'WA')]

In [47]:
# Flat map Transformation
airports.map(lambda c:(c[0], c[1])).filter(lambda c:c[1]=="WA").flatMap(lambda x: x).take(5)

['Bellingham', 'WA', 'Moses Lake', 'WA', 'Pasco']

In [48]:
# Distinct Transformation
airports.map(lambda c:(c[2])).distinct().take(5)

['Country', 'USA', 'Canada']

In [70]:
# Join Transformation
flt = flights.map(lambda c: (c[3], c[0]))
air = airports.map(lambda c: (c[3], c[1]))
flt.join(air).take(5)

[('ABE', ('01011245', 'PA')),
 ('ABE', ('01020600', 'PA')),
 ('ABE', ('01021245', 'PA')),
 ('ABE', ('01020605', 'PA')),
 ('ABE', ('01031245', 'PA'))]

In [60]:
# Zip with index
ac = airports.map(lambda c: (c[0], c[3]))
ac.zipWithIndex().take(5)
ac.zipWithIndex().filter(lambda x: x[1]>0).take(5)

[(('Abbotsford', 'YXX'), 1),
 (('Aberdeen', 'ABR'), 2),
 (('Abilene', 'ABI'), 3),
 (('Akron', 'CAK'), 4),
 (('Alamosa', 'ALS'), 5)]

In [65]:
# Reduce by key
(flights
 .zipWithIndex()
 .filter(lambda x: x[1]>0)
 .map(lambda row: row[0])
 .map(lambda c: (c[3], int(c[1])))
 .reduceByKey(lambda x, y: x + y)
 .take(5))

[('ABE', 5113), ('ACT', 392), ('ACV', 8429), ('ADQ', -254), ('AEX', 10193)]

In [66]:
# Sort by Key
(flights
 .zipWithIndex()
 .filter(lambda x: x[1]>0)
 .map(lambda row: row[0])
 .map(lambda c: (c[3], int(c[1])))
 .reduceByKey(lambda x, y: x + y)
 .sortByKey()
 .take(50))

[('ABE', 5113),
 ('ABI', 5128),
 ('ABQ', 64422),
 ('ABY', 1554),
 ('ACT', 392),
 ('ACV', 8429),
 ('ADQ', -254),
 ('AEX', 10193),
 ('AGS', 5003),
 ('ALB', 22362),
 ('ALO', 2866),
 ('AMA', 21979),
 ('ANC', 4948),
 ('ATL', 1151087),
 ('ATW', 8151),
 ('AUS', 108638),
 ('AVL', 5727),
 ('AVP', 2946),
 ('AZO', 233),
 ('BDL', 54662),
 ('BET', -645),
 ('BFL', 4022),
 ('BGR', 2852),
 ('BHM', 44355),
 ('BIL', 2616),
 ('BIS', 3825),
 ('BMI', 7817),
 ('BNA', 212243),
 ('BOI', 18004),
 ('BOS', 238602),
 ('BPT', 1936),
 ('BQK', 3952),
 ('BQN', 3943),
 ('BRO', 4967),
 ('BRW', 880),
 ('BTM', -138),
 ('BTR', 21989),
 ('BTV', 14755),
 ('BUF', 54309),
 ('BUR', 42241),
 ('BWI', 362845),
 ('BZN', 7226),
 ('CAE', 25686),
 ('CAK', 14749),
 ('CDC', 51),
 ('CDV', -1024),
 ('CEC', 2832),
 ('CHA', 7586),
 ('CHO', 2421),
 ('CHS', 30789)]

In [69]:
# Union Transformation
a = (
    airports
    .zipWithIndex()
    .filter(lambda row: row[1] > 0)
    .map(lambda row: row[0])
    .filter(lambda c: c[1] == "WA")
)
b = (
    airports
    .zipWithIndex()
    .filter(lambda row: row[1] > 0)
    .map(lambda row: row[0])
    .filter(lambda c: c[1] == "BC")
)
a.union(b).collect()

[['Bellingham', 'WA', 'USA', 'BLI'],
 ['Moses Lake', 'WA', 'USA', 'MWH'],
 ['Pasco', 'WA', 'USA', 'PSC'],
 ['Pullman', 'WA', 'USA', 'PUW'],
 ['Seattle', 'WA', 'USA', 'SEA'],
 ['Spokane', 'WA', 'USA', 'GEG'],
 ['Walla Walla', 'WA', 'USA', 'ALW'],
 ['Wenatchee', 'WA', 'USA', 'EAT'],
 ['Yakima', 'WA', 'USA', 'YKM'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Anahim Lake', 'BC', 'Canada', 'YAA'],
 ['Campbell River', 'BC', 'Canada', 'YBL'],
 ['Castlegar', 'BC', 'Canada', 'YCG'],
 ['Cranbrook', 'BC', 'Canada', 'YXC'],
 ['Fort Nelson', 'BC', 'Canada', 'YYE'],
 ['Fort Saint John', 'BC', 'Canada', 'YXJ'],
 ['Kamloops', 'BC', 'Canada', 'YKA'],
 ['Kelowna', 'BC', 'Canada', 'YLW'],
 ['Nanaimo', 'BC', 'Canada', 'YCD'],
 ['Penticton', 'BC', 'Canada', 'YYF'],
 ['Port Hardy', 'BC', 'Canada', 'YZT'],
 ['Powell River', 'BC', 'Canada', 'YPW'],
 ['Prince George', 'BC', 'Canada', 'YXS'],
 ['Prince Rupert', 'BC', 'Canada', 'YPR'],
 ['Quesnel', 'BC', 'Canada', 'YQZ'],
 ['"Sandspit, Queen Charlotte Islands"',

In [75]:
# Reduce action
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 1).reduce(lambda x,y:x/y)
print(data_reduce)

data_reduce1 = sc.parallelize([1, 2, .5, .1, 5, .2], 3).reduce(lambda x,y:x/y)
print(data_reduce1)

10.0
0.004


In [78]:
# Data frames
flightsDF = spark.read\
            .options(header='true', inferSchema='true')\
            .csv('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv')

In [79]:
flightsDF.createOrReplaceTempView('flightsDF')

In [80]:
spark.sql("select origin, sum(delay) as TotalDelay from flightsDF group by origin order by origin").show(50)

+------+----------+
|origin|TotalDelay|
+------+----------+
|   ABE|      5113|
|   ABI|      5128|
|   ABQ|     64422|
|   ABY|      1554|
|   ACT|       392|
|   ACV|      8429|
|   ADQ|      -254|
|   AEX|     10193|
|   AGS|      5003|
|   ALB|     22362|
|   ALO|      2866|
|   AMA|     21979|
|   ANC|      4948|
|   ATL|   1151087|
|   ATW|      8151|
|   AUS|    108638|
|   AVL|      5727|
|   AVP|      2946|
|   AZO|       233|
|   BDL|     54662|
|   BET|      -645|
|   BFL|      4022|
|   BGR|      2852|
|   BHM|     44355|
|   BIL|      2616|
|   BIS|      3825|
|   BMI|      7817|
|   BNA|    212243|
|   BOI|     18004|
|   BOS|    238602|
|   BPT|      1936|
|   BQK|      3952|
|   BQN|      3943|
|   BRO|      4967|
|   BRW|       880|
|   BTM|      -138|
|   BTR|     21989|
|   BTV|     14755|
|   BUF|     54309|
|   BUR|     42241|
|   BWI|    362845|
|   BZN|      7226|
|   CAE|     25686|
|   CAK|     14749|
|   CDC|        51|
|   CDV|     -1024|
|   CEC|      2832|


In [ ]:
# Abstracting data with data frames

In [81]:
sample_data = sc.parallelize([
      (1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD'
        , 13.75, 9.48, 0.61, 4.02)
    , (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD'
        , 11.04, 7.74, 0.52, 2.03)
    , (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD'
        , 12.8, 8.94, 0.68, 2.96)
    , (4, 'iMac', 2017, '27"', '64GB', '1TB SSD'
        , 25.6, 8.0, 20.3, 20.8)
])
sample_data_df = spark.createDataFrame(
    sample_data
    , [
        'Id'
        , 'Model'
        , 'Year'
        , 'ScreenSize'
        , 'RAM'
        , 'HDD'
        , 'W'
        , 'D'
        , 'H'
        , 'Weight'
    ]
)

In [84]:
sample_data_df.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [85]:
sample_data_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



In [87]:
import pyspark.sql as sql
import pyspark.sql.functions as f

sample_data_transformed = (sample_data_df.rdd
                           .map(lambda row:sql.Row(**row.asDict(), HDD_size = row.HDD.split(" ")[0]))
                           .map(lambda row:sql.Row(**row.asDict(), HDD_type = row.HDD.split(" ")[1]))
                           .map(lambda row:sql.Row(**row.asDict(), Volume=row.H * row.D * row.W))
                           .toDF()
                           .select (
                           sample_data_df.columns + [
                               'HDD_size',
                               'HDD_type',
                               f.round(f.col('Volume')).alias('Volume_cuIn')
                               ])
                        )
sample_data_transformed.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02, HDD_size='512GB', HDD_type='SSD', Volume_cuIn=80.0),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='12"', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03, HDD_size='256GB', HDD_type='SSD', Volume_cuIn=44.0)]

In [ ]:
#Infering schema using reflection

In [98]:
import pyspark.sql as sql
sample_data_rdd = sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv')
header = sample_data_rdd.first()
print(header)
sample_data_rdd_row = (sample_data_rdd
                       .filter(lambda row: row != header)
                       .map(lambda row: row.split(","))
                       .map(lambda row: 
                           sql.Row(
                               date = row[0],
                               delay = int(row[1]),
                               distance = float(row[2]),
                               origin = row[3],
                               destination = row[4]
                           ))
                      )
sample_data_rdd_row.take(4)
sample_data_rdd_row_df = spark.createDataFrame(sample_data_rdd_row)
sample_data_rdd_row_df.show(3)

date,delay,distance,origin,destination
+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|   602.0|   ABE|        ATL|
|01020600|   -8|   369.0|   ABE|        DTW|
|01021245|   -2|   602.0|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [101]:
#Specifiy schema programatically
import pyspark.sql as sql
import pyspark.sql.types as typ
sample_data_rdd = sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/departuredelays.csv')
header = sample_data_rdd.first()

sch = typ.StructType([
    typ.StructField('Date', typ.StringType(), False),
    typ.StructField('Delay', typ.LongType(), True),
    typ.StructField('distance', typ.LongType(), True),
    typ.StructField('origin', typ.StringType(), True),
    typ.StructField('destination', typ.StringType(), True)
])

sample_data_rdd = (sample_data_rdd
                   .filter(lambda row: row!=header)
                   .map(lambda row: row.split(","))
                   .map(lambda row:(
                       row[0],
                       int(row[1]),
                       int(row[2]),
                       row[3],
                       row[4]
                   ))
                  )

sample_data_with_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
sample_data_with_schema.show(3)

+--------+-----+--------+------+-----------+
|    Date|Delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [104]:
#Create temporary view
sample_data_with_schema.createOrReplaceTempView('sample_data_view')
spark.sql('''
select * from sample_data_view
''').show(10)

+--------+-----+--------+------+-----------+
|    Date|Delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [120]:
#Using SQL to interact with Dataframes
import pyspark.sql.types as typ

sch = typ.StructType([
      typ.StructField('Id', typ.LongType(), False)
    , typ.StructField('Model', typ.StringType(), True)
    , typ.StructField('Year', typ.IntegerType(), True)
    , typ.StructField('ScreenSize', typ.StringType(), True)
    , typ.StructField('RAM', typ.StringType(), True)
    , typ.StructField('HDD', typ.StringType(), True)
    , typ.StructField('W', typ.DoubleType(), True)
    , typ.StructField('D', typ.DoubleType(), True)
    , typ.StructField('H', typ.DoubleType(), True)
    , typ.StructField('Weight', typ.DoubleType(), True)
])
sample_data_rdd = sc.textFile('/Volumes/Data\ Disk/data_disk_workspace/pythonProjects/pySparkRecipes/data/sampleTextFile.txt')
header = sample_data_rdd.first()
sample_data_rdd = sample_data_rdd = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row: (
                int(row[0])
                , row[1]
                , int(row[2])
                , row[3]
                , row[4]
                , row[5]
                , float(row[6])
                , float(row[7])
                , float(row[8])
                , float(row[9])
        )
    )
)


sample_data_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
# sample_data_schema.show()
sample_data_schema.createOrReplaceTempView('sample_data_view')
# spark.sql('''
# select * from sample_data_view
# ''').show(10)

models_df = sc.parallelize([
      ('MacBook Pro', 'Laptop')
    , ('MacBook', 'Laptop')
    , ('MacBook Air', 'Laptop')
    , ('iMac', 'Desktop')
]).toDF(['Model', 'FormFactor'])

models_df.createOrReplaceTempView('models')

spark.sql('''
select a.*, b.FormFactor
from sample_data_view as a
left join models as b
on a.Model == b.Model
order by Weight desc
''').show()

#Aggregations
spark.sql('''
    SELECT b.FormFactor
        , COUNT(*) AS ComputerCnt
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    GROUP BY FormFactor
''').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|    Laptop|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+

+----------+-----------+
|FormFactor|ComputerCnt|
+----------+-----------+
|    Laptop|          3|
|   Desktop|          1|
+----------+-----------+



In [ ]:
# DAtaframes transformations
